In [3]:
import sqlite3

In [4]:
conn = sqlite3.connect("../data/aml_simulation.db")
cursor = conn.cursor()

In [5]:
# Rule 1: Structuring – multiple transfers just under threshold (e.g. 9,000 < x < 10,000)
cursor.execute("""
INSERT INTO flagged_txns (txn_id, rule_triggered, reason)
SELECT txn_id, 'Rule 1 - Structuring', 'Transaction amount between 9000 and 9999'
FROM transactions
WHERE amount BETWEEN 9000 AND 9999
""")

In [6]:
# Rule 2: Velocity – 5+ transactions from same account within 30 minutes
cursor.execute("""
INSERT INTO flagged_txns (txn_id, rule_triggered, reason)
SELECT txn_id, 'Rule 2 - High Velocity', 'More than 5 txns within 30 minutes'
FROM (
SELECT t.txn_id, t.account_id, t.timestamp,
COUNT(*) OVER (PARTITION BY t.account_id ORDER BY t.timestamp
RANGE BETWEEN 0 PRECEDING AND 1800 FOLLOWING) as txn_count
FROM transactions t
)
WHERE txn_count >= 5
""")

In [7]:
# Rule 3: High-risk destination countries
high_risk_countries = ('IR', 'RU', 'KP', 'SY', 'AF')
cursor.execute(f"""
INSERT INTO flagged_txns (txn_id, rule_triggered, reason)
SELECT txn_id, 'Rule 3 - High-Risk Country', 'Destination country is high-risk'
FROM transactions
WHERE dest_country IN {high_risk_countries}
""")

In [8]:
# Rule 4: Counterparty in Watchlist
cursor.execute("""
INSERT INTO flagged_txns (txn_id, rule_triggered, reason)
SELECT t.txn_id, 'Rule 4 - Watchlist Match', 'Counterparty matched watchlist entity'
FROM transactions t
JOIN watchlist_entities w ON t.counterparty = w.name
""")

In [9]:
# Rule 5: Dormant account suddenly active (no txns for >90 days, then sudden activity)
cursor.execute("""
INSERT INTO flagged_txns (txn_id, rule_triggered, reason)
WITH last_txns AS (
SELECT account_id, MAX(DATE(timestamp)) AS last_txn_date
FROM transactions
GROUP BY account_id
),
dormant_accounts AS (
SELECT a.account_id
FROM accounts a
JOIN last_txns l ON a.account_id = l.account_id
WHERE julianday('now') - julianday(l.last_txn_date) > 90
)
SELECT t.txn_id, 'Rule 5 - Dormant Reactivation', 'Dormant account reactivated'
FROM transactions t
JOIN dormant_accounts d ON t.account_id = d.account_id
""")

In [ ]:
conn.commit()
conn.close()